In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
#from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
import random

#from ggplot import *
#from scipy.stats import logistic
#from sklearn.metrics import roc_curve
#from sklearn.metrics import roc_auc_score
#from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import confusion_matrix, mean_squared_error
#from sklearn.model_selection import train_test_split
#from sklearn import preprocessing
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.naive_bayes import *
#from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
#import graphviz
#from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
#from sklearn.svm import SVC, SVR

# 한글 폰트 실행
from matplotlib import rcParams
from matplotlib import font_manager,rc
rcParams['axes.unicode_minus']=False
plt.rcParams['font.family']='NanumGothic'
font_name=font_manager.FontProperties(fname='c:/Windows/Fonts/malgun.ttf').get_name()
rc('font',family=font_name)


In [7]:
import time

def execution_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, rem = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(rem, 60)
        print(f"Execution time for {func.__name__}: {int(hours)}h {int(minutes)}m {seconds:.2f}s")
        return result
    return wrapper


@execution_time
def clean_df(data):
    data['고객ID'] = data['고객ID'].astype(str)
    data = reduce_mem_usage(data)
    print(data.info())
    return data


import pandas as pd 
import numpy as np

def reduce_mem_usage(df, float16_as32=True):
    # memory_usage()는 데이터프레임의 각 열의 메모리 사용량을 계산하고, sum은 그 값을 합산합니다. B -> KB -> MB로 변환합니다.
    start_mem = df.memory_usage().sum() / 1024**2
    print('데이터프레임의 초기 메모리 사용량은 {:.2f} MB입니다.'.format(start_mem))

    for col in df.columns:  # 각 열의 이름을 반복합니다.
        col_type = df[col].dtype  # 열의 데이터 타입을 가져옵니다.
        if col_type != object and str(col_type) != 'category':  # 객체 타입이 아닌 수치형 변수만 처리합니다.
            c_min, c_max = df[col].min(), df[col].max()  # 해당 열의 최소값과 최대값을 계산합니다.
            if str(col_type)[:3] == 'int':  # 정수형 변수일 경우 (int8, int16, int32, int64 모두 포함)
                # 이 열의 값 범위가 int8의 범위에 있을 경우 타입을 int8로 변환합니다. (-128부터 127까지)
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                # 이 열의 값 범위가 int16의 범위에 있을 경우 타입을 int16으로 변환합니다. (-32,768부터 32,767까지)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                # 이 열의 값 범위가 int32의 범위에 있을 경우 타입을 int32로 변환합니다. (-2,147,483,648부터 2,147,483,647까지)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                # 이 열의 값 범위가 int64의 범위에 있을 경우 타입을 int64로 변환합니다. (-9,223,372,036,854,775,808부터 9,223,372,036,854,775,807까지)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:  # 실수형일 경우
                # 값이 float16의 범위에 있을 경우 타입을 변환합니다. 더 높은 정밀도가 필요하면 float32를 고려합니다.
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    if float16_as32:  # 더 높은 정밀도가 필요하면 float32를 선택할 수 있습니다.
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float16)
                # 값이 float32의 범위에 있을 경우 타입을 float32로 변환합니다.
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                # 값이 float64의 범위에 있을 경우 타입을 float64로 변환합니다.
                else:
                    df[col] = df[col].astype(np.float64)
    # 최종 메모리 사용량을 계산합니다.
    end_mem = df.memory_usage().sum() / 1024**2
    print('최적화 후 데이터프레임의 메모리 사용량은: {:.2f} MB입니다.'.format(end_mem))
    # 초기 메모리 사용량과 비교하여 몇 퍼센트 감소했는지 계산합니다.
    print('메모리 사용량이 {:.1f}% 감소했습니다.'.format(100 * (start_mem - end_mem) / start_mem))

    return df

In [4]:
import pandas as pd
DATA_PATH = './dataset/CARD/'
DATA_PATH_2 = './dataset/CUSTOMER/'
DATA_PATH_3 = './dataset/CC/'
cc_22_40 = pd.read_csv(DATA_PATH_3 + 'cc_22_40.csv', encoding="utf-8-sig")
cc_22_50 = pd.read_csv(DATA_PATH_3 + 'cc_22_50.csv', encoding="utf-8-sig")
cc_22_60 = pd.read_csv(DATA_PATH_3 + 'cc_22_60.csv', encoding="utf-8-sig")

In [5]:
cc_22_40.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076734 entries, 0 to 14076733
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      int64 
 1   거래년월      int64 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int64 
dtypes: int64(3), object(4)
memory usage: 751.8+ MB


In [6]:
cc_22_40.head()

,고객ID,거래년월,가맹점업종명,가맹점_광역시도,가맹점_시군구,승인건수,승인금액
0,9997364303,202212,이동통신요금,서울특별시,중구,5번이하,70000
1,9997364303,202212,외식,경상북도,영천시,5번이하,120000
2,9997364303,202212,온라인쇼핑,경기도,성남시 분당구,5번이하,80000
3,9997364303,202212,의료/보험,경상북도,영천시,5번이하,10000
4,9997364303,202212,외식,경상북도,영천시,5번이하,30000


In [8]:
cc_22_40=clean_df(cc_22_40)

데이터프레임의 초기 메모리 사용량은 751.78 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 644.38 MB입니다.
메모리 사용량이 14.3% 감소했습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076734 entries, 0 to 14076733
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      object
 1   거래년월      int32 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 644.4+ MB
None
Execution time for clean_df: 0h 0m 4.59s


In [11]:
cc_22_40_new = reduce_mem_usage(cc_22_40)

데이터프레임의 초기 메모리 사용량은 644.38 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 644.38 MB입니다.
메모리 사용량이 0.0% 감소했습니다.


In [12]:
cc_22_40_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14076734 entries, 0 to 14076733
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      object
 1   거래년월      int32 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 644.4+ MB


In [14]:
cc_22_50=clean_df(cc_22_50)
cc_22_50_new = reduce_mem_usage(cc_22_50)
cc_22_50_new.info()

데이터프레임의 초기 메모리 사용량은 745.38 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 745.38 MB입니다.
메모리 사용량이 0.0% 감소했습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16283134 entries, 0 to 16283133
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      object
 1   거래년월      int32 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 745.4+ MB
None
Execution time for clean_df: 0h 0m 1.03s
데이터프레임의 초기 메모리 사용량은 745.38 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 745.38 MB입니다.
메모리 사용량이 0.0% 감소했습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16283134 entries, 0 to 16283133
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      object
 1   거래년월      int32 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 745.4+ MB


In [15]:
cc_22_60=clean_df(cc_22_60)
cc_22_60_new = reduce_mem_usage(cc_22_60)
cc_22_60_new.info()

데이터프레임의 초기 메모리 사용량은 644.61 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 552.52 MB입니다.
메모리 사용량이 14.3% 감소했습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12069980 entries, 0 to 12069979
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      object
 1   거래년월      int32 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 552.5+ MB
None
Execution time for clean_df: 0h 0m 3.86s
데이터프레임의 초기 메모리 사용량은 552.52 MB입니다.
최적화 후 데이터프레임의 메모리 사용량은: 552.52 MB입니다.
메모리 사용량이 0.0% 감소했습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12069980 entries, 0 to 12069979
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   고객ID      object
 1   거래년월      int32 
 2   가맹점업종명    object
 3   가맹점_광역시도  object
 4   가맹점_시군구   object
 5   승인건수      object
 6   승인금액      int32 
dtypes: int32(2), object(5)
memory usage: 552.5+ MB


In [16]:
cc_22_40_new.to_csv('cc_20_40.csv', index=False, encoding='utf-8-sig')
cc_22_50_new.to_csv('cc_20_50.csv', index=False, encoding='utf-8-sig')
cc_22_60_new.to_csv('cc_20_60.csv', index=False, encoding='utf-8-sig')